In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
from tqdm import trange


from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from keras import regularizers
import keras
from keras import optimizers
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import adam_v2
from keras.callbacks import ModelCheckpoint
import random
import math
import matplotlib.image as mpimg
import cv2
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras import callbacks
from keras import layers, models

from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50

from google.colab.patches import cv2_imshow

In [ ]:
#loading and preprocessing the images
def create_dataset(path):
  img_data_array = []

  for i in trange(1, 7981):
            
        path2 = "img"+ str(i) +".jpg"
        path1 = os.path.join(path, path2)
        image= cv2.imread(path1, cv2.COLOR_BGR2RGB)
        image=cv2.resize(image, (224, 224),interpolation = cv2.INTER_AREA)
        image=np.array(image)
        image = image.astype('float32')
        image /= 255 
        img_data_array.append(image)

  return img_data_array


# extract the image array and class name
img_data=create_dataset('x14x16x18/x14x16x18-images')     #check if the images are mapped correctly to labels

100%|██████████| 7980/7980 [39:11<00:00,  3.39it/s]


In [ ]:
train_data = np.asarray(img_data)
train_data.shape

(7980, 224, 224, 3)

In [ ]:
y_train = pd.read_csv('x14x16x18/x141618-actuation-labels.csv') #reading the labels file

In [ ]:
num_classes = 5 #5 classes coreesponding to the 5 actuations

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data, y_train, test_size = 0.3, random_state=600, shuffle=True)  #shuffling and splitting the data into train and test sets

In [ ]:
input_shape = (224, 224, 3) #setting image input shape to 224x224

In [ ]:
#scaling the labels to lie between 0 and 1
target_scaler = MinMaxScaler()  
target_scaler.fit(y_train)
y_train = target_scaler.transform(y_train)
y_test = target_scaler.transform(y_test)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
#import and instantiate the optimizer
from keras.optimizer_v2.adam import Adam

adam = Adam()

In [ ]:

new_input = Input(shape=(224, 224, 3))
vgg_model = VGG16(include_top=False, weights='imagenet', input_tensor=new_input)

for layer in vgg_model.layers[:13]:
  layer.trainable = False

for layer in vgg_model.layers:
    print(layer, layer.trainable)


58900480/58889256 [==============================] - 0s 0us/step
<keras.engine.input_layer.InputLayer object at 0x7fc1f6221550> False
<keras.layers.convolutional.Conv2D object at 0x7fc1f6b2d2d0> False
<keras.layers.convolutional.Conv2D object at 0x7fc1f59b4f90> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc1e027a510> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e027dd10> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e0285310> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc1f59f1190> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e028a190> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e028eb90> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e0291310> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc1e028a9d0> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e029ccd0> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e02a3390> False
<keras.layers.convolutional.Conv2D object at 0x7fc1e028e810> T

In [ ]:
#defining and creating the model 
def create_model():
  model = Sequential()
  model.add(vgg_model)
  model.add(layers.Flatten())
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0005)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0005)))
  model.add(layers.Dropout(0.25))
  model.add(layers.BatchNormalization())

  model.add(layers.Dense(5, activation='linear', kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0005)))
  model.add(layers.Dropout(0.4))
  model.compile(optimizer= Adam(0.0003),#"adam",
              loss='mean_squared_error',
              metrics=['mse'])
  return model


In [ ]:
# new_model = keras.models.load_model('x14x16-newbg/x14x16x18-model-3.h5')  #loading previously saved model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 25088)             100352    
_________________________________________________________________
dense (Dense)                (None, 64)                1605696   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [ ]:
#time-based learning rate scheduler
initial_learning_rate = 0.01
epochs = 150
decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [ ]:
#training
history = model.fit(x_train, y_train,
          epochs=150,
          batch_size = 128,
          validation_split=0.3,
          callbacks=[callbacks.LearningRateScheduler(lr_time_based_decay, verbose=1)],
          verbose=1)

Epoch 1/150

Epoch 00001: LearningRateScheduler setting learning rate to 0.0003000000142492354.
31/31 [==============================] - 47s 641ms/step - loss: 3.1604 - mse: 1.8960 - val_loss: 21.2671 - val_mse: 20.0681
Epoch 2/150

Epoch 00002: LearningRateScheduler setting learning rate to 0.00029998001558153.
31/31 [==============================] - 12s 377ms/step - loss: 2.5241 - mse: 1.3537 - val_loss: 5.2957 - val_mse: 4.1476
Epoch 3/150

Epoch 00003: LearningRateScheduler setting learning rate to 0.00029994002791398976.
31/31 [==============================] - 12s 377ms/step - loss: 2.3729 - mse: 1.2413 - val_loss: 2.7056 - val_mse: 1.5916
Epoch 4/150

Epoch 00004: LearningRateScheduler setting learning rate to 0.00029988005524361533.
31/31 [==============================] - 12s 376ms/step - loss: 2.2260 - mse: 1.1279 - val_loss: 2.1435 - val_mse: 1.0629
Epoch 5/150

Epoch 00005: LearningRateScheduler setting learning rate to 0.00029980010156634173.
31/31 [======================

In [ ]:
#  model.evaluate(x_test, y_test) #evaluate on test set

In [ ]:
#prediction on test set
prediction = model.predict(x_test)
prediction = target_scaler.inverse_transform(prediction)
prediction = np.rint(prediction)

In [ ]:
print(prediction)

[[ 17.  -2.  -3.  14.  16.]
 [ 16.  -9.   0.  15.  18.]
 [ 18. -12.  -4.  14.  17.]
 ...
 [ 14. -12.  -1.  16.  15.]
 [ 18. -16.  -3.  14.  17.]
 [ 16. -13.   0.  16.  16.]]


In [ ]:
y_test = target_scaler.inverse_transform(y_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#calculating mse and mae between prediction and the ground truth values
mse = mean_squared_error(y_test, prediction)
mae = mean_absolute_error(y_test, prediction)

In [ ]:
print(mse)
print(mae)

23.076274018379284
3.0027568922305763


In [ ]:
model.save("x14x16-model-1")  #saving the model

In [ ]:
model.save("newbg-141618-75-model.h5")  #saving the model in h5 format